In [1]:
from lazypredict.Supervised import LazyClassifier

/Users/sharabh_shukla/opt/anaconda3/envs/py_devops/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('../data/preprocessed/seeds_data.csv', index_col=[0])

In [5]:
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
preprocessor_pipe = Pipeline([
    ('power_transformer', PowerTransformer())
])

X = preprocessor_pipe.fit_transform(data.drop(columns=['target'], axis=1).to_numpy())
Y = data['target'].to_numpy()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=123)

In [7]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)

In [8]:
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 30/30 [00:00<00:00, 47.03it/s]

[22:54:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [9]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LinearSVC,0.97,0.98,None,0.98,0.01
LinearDiscriminantAnalysis,0.97,0.98,None,0.98,0.01
XGBClassifier,0.97,0.98,None,0.98,0.06
SGDClassifier,0.97,0.98,None,0.98,0.01
RidgeClassifierCV,0.97,0.98,None,0.98,0.01
RidgeClassifier,0.97,0.98,None,0.98,0.01
RandomForestClassifier,0.97,0.98,None,0.98,0.11
PassiveAggressiveClassifier,0.97,0.98,None,0.98,0.01
NearestCentroid,0.97,0.98,None,0.98,0.01


In [11]:
from mlxtend.classifier import StackingCVClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression, RidgeClassifierCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

classifiers = (LinearDiscriminantAnalysis(), RidgeClassifierCV(), LinearSVC())

In [12]:
stacked_classifier = StackingCVClassifier(classifiers=classifiers, meta_classifier=RandomForestClassifier())

In [13]:
from sklearn.pipeline import Pipeline
clf_pipe = Pipeline([
    ('power_transformer', PowerTransformer()),
    ('stacked_classifier', stacked_classifier)
])

In [14]:
clf_pipe.fit(X_train, y_train)


Pipeline(steps=[('power_transformer', PowerTransformer()),
                ('stacked_classifier',
                 StackingCVClassifier(classifiers=(LinearDiscriminantAnalysis(),
                                                   RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ])),
                                                   LinearSVC()),
                                      meta_classifier=RandomForestClassifier()))])

In [15]:
clf_pipe.score(X_test, y_test)

0.975

In [16]:
y_pred = clf_pipe.predict(X_test)

In [17]:
pred_df = pd.DataFrame({'Y-predictions': y_pred, 'Y-test': y_test})

In [18]:
pred_df

,Y-predictions,Y-test
0,3,3
1,1,1
2,2,2
3,1,1
4,1,1
5,1,1
6,3,3
7,2,2
8,1,1
9,3,3


In [19]:
from joblib import dump

In [20]:
dump(clf_pipe, '../models/clf_pipe')

['../models/clf_pipe']

In [21]:
x_prediction_test = np.random.rand(100,7)

In [22]:
clf_pipe.predict(x_prediction_test)

array([1, 2, 2, 1, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 3, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1,
       2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       2, 1, 2, 2, 2, 1, 1, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2])